In [124]:
import numpy as np
import cv2
import os
import pandas as pd
import h5py

In [125]:
train_folder = "./train"
test_folder = "./test"
extra_folder = "./extra"
resize_size = (64, 64)

In [126]:
def collapse_col(row):
    global resize_size
    new_row = {}
    new_row['img_name'] = list(row['img_name'])[0]
    new_row['labels'] = row['label'].astype(np.str).str.cat(sep='_')
    new_row['top'] = max(int(row['top'].min()), 0)
    new_row['left'] = max(int(row['left'].min()), 0)
    new_row['bottom'] = int(row['bottom'].max())
    new_row['right'] = int(row['right'].max())
    new_row['width'] = int(new_row['right'] - new_row['left'])
    new_row['height'] = int(new_row['bottom'] - new_row['top'])
    new_row['num_digits'] = len(row['label'].values)
    return pd.Series(new_row, index=None)

In [127]:
def get_name(index, hdf5_data):
    name = hdf5_data['/digitStruct/name']
    return ''.join([chr(v[0]) for v in hdf5_data[name[index][0]].value])

In [128]:
def get_bbox(index, hdf5_data):
    attrs = {}
    item = hdf5_data['digitStruct']['bbox'][index].item()
    for key in ['label', 'left', 'top', 'width', 'height']:
        attr = hdf5_data[item][key]
        values = [hdf5_data[attr.value[i].item()].value[0][0]
                  for i in range(len(attr))] if len(attr) > 1 else [attr.value[0][0]]
        attrs[key] = values
    return attrs

In [129]:
def getFilePathInfo(filepath):

    dirname = os.path.dirname(filepath)
    basename = os.path.basename(filepath)
    info = os.path.splitext(basename)
    filename = info[0]
    extend = info[1]

    return dirname, filename, extend

In [130]:
def img_boundingbox_data_constructor(img_folder, mat_file):
    mat_file = os.path.join(img_folder, mat_file)
    info = getFilePathInfo(mat_file)
    bbox_csv_filepath = os.path.join(info[0], 'bbox.csv')
    print(bbox_csv_filepath)

    if os.path.isfile(bbox_csv_filepath):
        print("bounding box file: {} exists, read from file".format(bbox_csv_filepath))
        bbox_df = pd.read_csv(bbox_csv_filepath)
        # print(bbox_df)
        print('finished image bounding box data construction...')
        return bbox_df

    f = h5py.File(mat_file, 'r')
    all_rows = []
    print('image bounding box data construction starting...')
    bbox_df = pd.DataFrame([], columns=['img_name', 'img_width', 'img_height',
                                        'label',
                                        'top', 'bottom', 'left', 'right', 'width', 'height',
                                        'rel_x_center', 'rel_y_center', 'rel_width', 'rel_height'])

    i = 0
    for j in range(f['/digitStruct/bbox'].shape[0]):
        img_name = get_name(j, f)
        row_dict = get_bbox(j, f)
        row_dict['img_name'] = img_name
        img = cv2.imread(os.path.join(img_folder, img_name))
        row_dict['img_height'], row_dict['img_width'], _ = img.shape
        # map class id 10 to class id 0 for digit 0
        row_dict['label'] = [int(l)%10 for l in row_dict['label']]
        if (i % 1000) == 0:
            print(row_dict)
        all_rows.append(row_dict)
        bbox_df = pd.concat([bbox_df, pd.DataFrame.from_dict(row_dict, orient='columns')])
        # if j > 20:
        #     break
        i += 1

    bbox_df['bottom'] = bbox_df['top'] + bbox_df['height']
    bbox_df['right'] = bbox_df['left'] + bbox_df['width']
    bbox_df['rel_x_center'] = (bbox_df['left'] + bbox_df['right']) / 2 / bbox_df['img_width']
    bbox_df['rel_y_center'] = (bbox_df['top'] + bbox_df['bottom']) / 2 / bbox_df['img_height']
    bbox_df['rel_width'] = bbox_df['width'] / bbox_df['img_width']
    bbox_df['rel_height'] = bbox_df['height'] / bbox_df['img_height']

    bbox_df.to_csv(bbox_csv_filepath, index=False)
    print(bbox_df)

    print('finished image bounding box data construction...')
    return bbox_df

In [131]:
def output_yolov4_label_txt(img_folder, img_name, output_line):
    label_name = os.path.splitext(img_name)[0] + '.txt'
    f = open(os.path.join(img_folder, label_name), 'a+')
    f.write(output_line)
    f.write('\n')
    f.close()

In [132]:
def construct_all_data(img_folder, mat_file_name, h5_name):
    img_bbox_data = img_boundingbox_data_constructor(img_folder, mat_file_name)
    print("\nimg_bbox_data:\n{}".format(img_bbox_data))

    img_bbox_data_list = img_bbox_data.to_dict(orient='records')
    # print(img_bbox_data_list)

    delete_label_txt = 'rm {}/*.txt'.format(os.path.join(img_folder))
    os.system(delete_label_txt)

    # for row in img_bbox_data_list[0:10]:
    for row in img_bbox_data_list:
        output_line = '{} {} {} {} {}'.format(row['label'], row['rel_x_center'], row['rel_y_center'], row['rel_width'], row['rel_height'])
        output_yolov4_label_txt(img_folder, row['img_name'], output_line)

In [136]:
construct_all_data(train_folder, 'digitStruct.mat', 'train_data_processed.h5')

./train/bbox.csv
image bounding box data construction starting...
{'label': [1, 9], 'left': [246.0, 323.0], 'top': [77.0, 81.0], 'width': [81.0, 96.0], 'height': [219.0, 219.0], 'img_name': '1.png', 'img_height': 350, 'img_width': 741}


<ipython-input-127-f68001855196>:3: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  return ''.join([chr(v[0]) for v in hdf5_data[name[index][0]].value])
<ipython-input-128-122470932d59>:6: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  values = [hdf5_data[attr.value[i].item()].value[0][0]
<ipython-input-128-122470932d59>:7: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  for i in range(len(attr))] if len(attr) > 1 else [attr.value[0][0]]


{'label': [6], 'left': [36.0], 'top': [4.0], 'width': [14.0], 'height': [31.0], 'img_name': '1001.png', 'img_height': 42, 'img_width': 89}
{'label': [3], 'left': [37.0], 'top': [5.0], 'width': [14.0], 'height': [34.0], 'img_name': '2001.png', 'img_height': 47, 'img_width': 86}
{'label': [3, 9], 'left': [66.0, 90.0], 'top': [13.0, 14.0], 'width': [28.0, 31.0], 'height': [72.0, 72.0], 'img_name': '3001.png', 'img_height': 95, 'img_width': 186}
{'label': [6, 1], 'left': [23.0, 38.0], 'top': [3.0, 4.0], 'width': [14.0, 7.0], 'height': [25.0, 25.0], 'img_name': '4001.png', 'img_height': 33, 'img_width': 72}
{'label': [3, 4, 7], 'left': [65.0, 78.0, 87.0], 'top': [32.0, 32.0, 27.0], 'width': [16.0, 12.0, 14.0], 'height': [30.0, 30.0, 30.0], 'img_name': '5001.png', 'img_height': 78, 'img_width': 166}
{'label': [5, 6], 'left': [56.0, 74.0], 'top': [21.0, 21.0], 'width': [19.0, 17.0], 'height': [39.0, 39.0], 'img_name': '6001.png', 'img_height': 76, 'img_width': 158}
{'label': [6, 0], 'left': [

In [137]:
obj_folder = '../build/darknet/x64/data/obj'
obj_folder_rel2_darknet = './build/darknet/x64/data/obj'
files = [int(os.path.splitext(f)[0]) for f in os.listdir(obj_folder) if isfile(join(obj_folder, f)) and os.path.splitext(f)[1]=='.png']
files.sort()
# print(files)

train_test_split_ratio = 20
train_list = ['{}/{}.png\n'.format(obj_folder_rel2_darknet, f) for f in files if not (f%train_test_split_ratio)==0]
# print(train_list)

test_list = ['{}/{}.png\n'.format(obj_folder_rel2_darknet, f) for f in files if (f%train_test_split_ratio)==0]
# print(test_list)

In [138]:
list_folder = '../build/darknet/x64/data'
train_list_filename = 'train.txt'
test_list_filename = 'test.txt'

fp = open(os.path.join(list_folder, train_list_filename), 'w')
fp.writelines(train_list)
fp.close()

fp = open(os.path.join(list_folder, test_list_filename), 'w')
fp.writelines(test_list)
fp.close()
